In [ ]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.3'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-12-01 04:21:01--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2022-12-01 04:21:04 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Sportsproducts").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

sports_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
sports_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
# number of reviews

print(sports_df.count())

4850360


In [ ]:
# drop na and duplicates

sports_df = sports_df.dropna()
print(sports_df.count())

new_sports_df = sports_df.dropDuplicates()
print(new_sports_df.count())

4849000
4849000


In [ ]:
# schema
new_sports_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# create review dataframe
sports_review_df = new_sports_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])

sports_review_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100E6OT70LKOZ|   21498849|B001E29RG6|     221089601| 2011-02-27|
|R100JZDN9RV012|   28299024|B0035HEPXS|     173761747| 2011-10-05|
|R100WJRDBA7IQ5|   40179799|B0019GJZXW|     704428554| 2014-06-18|
|R100ZY64ZJVNVZ|   25717098|B000MMZ5ZG|     247901753| 2015-02-23|
|R101GPEDWLA42J|   42195961|B00U821IME|     647954521| 2015-05-09|
|R101ME0S73N6VJ|   22434214|B007EIET44|     651071819| 2014-02-17|
|R101WXYFUIKM3L|   26183682|B0009PVUW8|     809898756| 2012-06-28|
|R1025ZMLB7LVOB|    2805819|B00L5QZ4YU|     221207062| 2014-12-12|
|R1026YY3Y2Y90K|    1032610|B001PR0NVS|     863659872| 2015-01-30|
|R102ORVDLW2JPZ|   14066424|B008L5Y0O8|     495880211| 2015-03-14|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# unique values dataframe
sports_products_df = new_sports_df.select(["product_id", "product_title"])
sports_products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001E29RG6|Dallas Cowboys Lu...|
|B0035HEPXS|Smooth Fitness 5....|
|B0019GJZXW|SKLZ Baseball Spe...|
|B000MMZ5ZG|Cannon Low-Profil...|
|B00U821IME|5 Pcs Assort Agat...|
|B007EIET44| Cressi Silicone Cap|
|B0009PVUW8|Park & Sun Sports...|
|B00L5QZ4YU|UM Tactical UM3 S...|
|B001PR0NVS|NFL Embossed Genu...|
|B008L5Y0O8|Under Armour Antl...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# customer dataframe

# number of customers
customer_count = new_sports_df.select(["customer_id"]).count()

customer_count

4849000

In [ ]:
# dataframe
sports_customers_df = new_sports_df.groupBy("customer_id").count()
sports_customers_df = sports_customers_df.withColumnRenamed("count", "customer_count")
sports_customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1513285|             7|
|    1924038|             7|
|    5625756|             3|
|   48333068|             7|
|   37443191|             4|
|   11280609|             3|
|   21867585|             5|
|   10497342|             3|
|   21191157|             1|
|   22573812|             2|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# vine data frame

sports_vine_df = new_sports_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

sports_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100E6OT70LKOZ|          3|            0|          0|   N|
|R100JZDN9RV012|          5|            1|          2|   N|
|R100WJRDBA7IQ5|          5|            0|          0|   N|
|R100ZY64ZJVNVZ|          5|            0|          0|   N|
|R101GPEDWLA42J|          4|            0|          0|   N|
|R101ME0S73N6VJ|          5|            0|          0|   N|
|R101WXYFUIKM3L|          5|            1|          1|   N|
|R1025ZMLB7LVOB|          5|            7|          7|   N|
|R1026YY3Y2Y90K|          5|            0|          0|   N|
|R102ORVDLW2JPZ|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows

